#  agentic workflow with haystack
load the company information in postprocessing.csv, 

add industry, founder background like ex-faang, competitive programmer, 10 years experience, serial founder, ...



In [1]:

from haystack import Pipeline, Document, component
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack_integrations.components.generators.ollama import OllamaGenerator
from haystack.dataclasses import ChatMessage
from typing import Optional, List
from pydantic import BaseModel, ConfigDict
from pprint import pprint
import json
import matplotlib.pyplot as plt
import pandas as pd
import os
import copy
from tqdm import tqdm 

In [ ]:
path_to_scrapy_output = '../yc_scraper/output.json'

assert os.path.exists(path_to_scrapy_output), f'path not found: {path_to_scrapy_output}'

with open(path_to_scrapy_output, 'r') as file:
    data = json.load(file) 
    print('data len = ', len(data))

company = data[0]

In [ ]:
pprint(company)

In [14]:
#define model to b used
# OLLAMA_MODEL = "deepseek-r1:8b"
# OLLAMA_MODEL = "deepseek-r1:1.5b"
# OLLAMA_MODEL = "deepseek-r1:1.5b"
# OLLAMA_MODEL = "phi4:latest"
OLLAMA_MODEL = "llama3.2:3b"

MAX_COMPONENT_RUN = 10

# Define template
template = "Answer the question: {{query}}"

# Initialize pipeline
pipe = Pipeline()

# Add components
generator = OllamaGenerator(model=OLLAMA_MODEL, url="http://localhost:11434")
pipe.add_component("prompt_builder", PromptBuilder(template=template))
pipe.add_component("llm", generator)

# Connect components correctly
pipe.connect("prompt_builder", "llm.prompt")

# Run pipeline
query = "Who is Super Mario? Give me a short answer within 10 words"
res = pipe.run({"prompt_builder": {"query": query}})

# Print results
pprint(res['llm']['replies'][0])

'Super Mario is the iconic video game mascot and plumber.'


In [15]:
# Validator
class Answer(BaseModel):
    keywords: List[str]
    model_config = ConfigDict(extra='forbid')

json_schema = Answer.model_json_schema()
pprint(json_schema)

{'additionalProperties': False,
 'properties': {'keywords': {'items': {'type': 'string'},
                             'title': 'Keywords',
                             'type': 'array'}},
 'required': ['keywords'],
 'title': 'Answer',
 'type': 'object'}


In [16]:
@component
class OutputValidator:

    def __init__(self, pydantic_model: BaseModel):
        self.pydantic_model = pydantic_model
        self.iter_count = 0
    
    @component.output_types(valid_replies = List[str], invalid_replies = Optional[List[str]], error_msg = Optional[str])
    def run(self, replies: List[str]):
        self.iter_count += 1
        # print('replies = ', replies)
        try:
            output_dict = json.loads(replies[0])
            # assert len(output_dict['keywords']) == 10, "Keywords must contain 10"
            self.pydantic_model.model_validate(output_dict)
            # print(f'[OK], iter_count={self.iter_count} valid')
            return {'valid_replies': replies}
        except ValueError as e:
            # print(f'[NOT OK] format not right')
            return {'invalid_replies': replies, 'error_msg': str(e)}
        # except AssertionError as e:
        #     print('f[NOT OK]: {e}')
        #     return {'invalid_replies': replies, 'error_msg': str(e)}



In [17]:
def extract_valid_replies(res):
    return json.loads(res['json_validator']['valid_replies'][0])

In [18]:
template = '''
Extract exactly 10 keywords about this company industry from this company description: {{company_description}}. Exclude any company names. Keep the keywords lowercase
Only use information given in the company description, Follow this JSON schema, but only return the actual instances without any additional schema definition: {{schema}}.
Make sure you response is a dict, not a list
{% if invalid_replies and error_message %}
  Read the instruction carefully and answer differently from this {{invalid_replies}}
  becuase that answer format got this Python exception: {{error_message}}
  Fix the above Python exception and return the corrected output without any extra explanations.
{% endif %}
'''


company_pipeline = Pipeline(max_runs_per_component=MAX_COMPONENT_RUN) 
company_pipeline.add_component(name="prompt_builder", instance=PromptBuilder(template = template))
company_pipeline.add_component(name="llm", instance = OllamaGenerator(model=OLLAMA_MODEL, url = 'http://localhost:11434'))
company_pipeline.add_component(name="json_validator", instance=OutputValidator(pydantic_model = Answer))
company_pipeline.connect("prompt_builder", "llm")
company_pipeline.connect("llm.replies", "json_validator")
company_pipeline.connect("json_validator.invalid_replies", 'prompt_builder.invalid_replies')
company_pipeline.connect("json_validator.error_msg", 'prompt_builder.error_message')

# company_pipeline.show()

res = company_pipeline.run({'prompt_builder': {'company_description': company['description'], 'schema': json_schema}})

/home/nguyen/anaconda3/envs/torch/lib/python3.12/site-packages/haystack/core/pipeline/pipeline.py:523: RuntimeWarning: Pipeline is stuck running in a loop. Partial outputs will be returned. Check the Pipeline graph for possible issues.
  warn(RuntimeWarning(msg))


In [19]:
pprint(extract_valid_replies(res))
print(type(extract_valid_replies(res)))

{'keywords': ['oil',
              'gas',
              'emission',
              'methane',
              'finance',
              'investment',
              'regulation',
              'technology']}
<class 'dict'>


In [20]:
pprint(company['founders'])

[{'background': 'Co-Founder Orbio Earth GmbH; Ex-ML Engineer NASA Harvest; '
                'Ex-Cervest',
  'name': 'Robert Huppertz'},
 {'background': 'Climate optimist building out product and commercial strategy '
                '@ Orbio Earth',
  'name': 'Jack Angela'}]


In [21]:
founder_template = '''
from this person background: {{founder_background}} 
Extract at least 5 most important keywords about this person and put all of it in keywords, 
Determine if this person is technical or non-technical and add to keywords,
If this person has worked in a big company, add company to keywords,
if this person has education from ivy league schools, add ivyleague in keywords,  
# if this person has founded startups before, add serial founders, 
Only use information given in the background, Exactly follow this JSON schema: {{schema}}.
Only return the actual instances, no extra attributes in the schema is allowed,
Keep each keyword short, lowercase, and no extra space or newline,
ke sure you response is a dict, not a list
{% if invalid_replies and error_message %}
  Read the instruction carefully and answer differently from this {{invalid_replies}}
  becuase that answer format got this Python exception: {{error_message}}
  Fix the above Python exception and return the corrected output without any extra explanations.
{% endif %}
'''


founder_pipeline = Pipeline(max_runs_per_component=MAX_COMPONENT_RUN) 
founder_pipeline.add_component(name="prompt_builder", instance=PromptBuilder(template = founder_template))
founder_pipeline.add_component(name="llm", instance = OllamaGenerator(model=OLLAMA_MODEL, url = 'http://localhost:11434'))
founder_pipeline.add_component(name="json_validator", instance=OutputValidator(pydantic_model = Answer))
founder_pipeline.connect("prompt_builder", "llm")
founder_pipeline.connect("llm.replies", "json_validator")
founder_pipeline.connect("json_validator.invalid_replies", 'prompt_builder.invalid_replies')
founder_pipeline.connect("json_validator.error_msg", 'prompt_builder.error_message')

for founder in company['founders']:
  founder_res = founder_pipeline.run({'prompt_builder': {'founder_background': founder['background'], 'schema': json_schema}})
  res = extract_valid_replies(founder_res)
  print('final answer = ', res)


final answer =  {'keywords': ['tech', 'ml', 'nasa', 'startup', 'engineer']}


PipelineMaxComponentRuns: Maximum run count 10 reached for component 'prompt_builder'

In [13]:
# res = extract_valid_replies(founder_res)
# print(res)
# print(type(res))

# Run the pipeline and add to final dataset

In [23]:
def extract_company_founders_keywords(datapoint, company_pipeline, founder_pipeline, self_correct_iter = 5):
    newdata = copy.deepcopy(datapoint)

    for iter in range(1, self_correct_iter + 1):
        try:

            company_keywords_res = company_pipeline.run(
                {'prompt_builder': 
                    {
                        'company_description': newdata['description'], 
                        'schema': json_schema}
                    })
                
            company_keywords = extract_valid_replies(company_keywords_res)

            founders = newdata['founders']
            founder_keywords = set()

            for founder in founders:
                founder_keywords_res = founder_pipeline.run(
                    {'prompt_builder': 
                        {
                            'founder_background': founder['background'], 
                            'schema': json_schema}
                        }
                    )
                founder_keywords.update(extract_valid_replies(founder_keywords_res)['keywords'])
            
            newdata['company_keywords'] = company_keywords
            newdata['founder_keywords'] = list(founder_keywords)
            return newdata 
        except Exception as e:
            print(f'iter {iter} failed, trying again {iter}/{self_correct_iter}, {e}')
    
    raise AssertionError(f"failed {self_correct_iter} to validate output format")

In [24]:
new_data = []
error_data = []
loop = tqdm(data)
for datapoint in loop:
    try:
        new_datapoint = extract_company_founders_keywords(datapoint, company_pipeline, founder_pipeline)
        new_data.append(new_datapoint)
    
    except Exception as e:
        error_data.append(datapoint)
    loop.set_description(f"error data = {len(error_data)}, error_rate = {(len(error_data) / len(new_data))*100 if len(new_data)!=0 else 0:.2f}%")


  0%|          | 0/3096 [00:00<?, ?it/s]/home/nguyen/anaconda3/envs/torch/lib/python3.12/site-packages/haystack/core/pipeline/pipeline.py:523: RuntimeWarning: Pipeline is stuck running in a loop. Partial outputs will be returned. Check the Pipeline graph for possible issues.
  warn(RuntimeWarning(msg))
error data = 0, error_rate = 0.00:   0%|          | 3/3096 [00:06<1:56:11,  2.25s/it]

Maximum run count 10 reached for component 'prompt_builder'
iter 0 failed, trying again 0/5


error data = 0, error_rate = 0.00:   0%|          | 7/3096 [00:27<2:55:05,  3.40s/it]

Maximum run count 10 reached for component 'prompt_builder'
iter 0 failed, trying again 0/5
Maximum run count 10 reached for component 'prompt_builder'
iter 1 failed, trying again 1/5


error data = 0, error_rate = 0.00:   0%|          | 14/3096 [01:09<3:11:07,  3.72s/it]

...

error data = 85, error_rate = 2.83: 100%|█████████▉| 3090/3096 [5:39:08<00:35,  5.98s/it]

Maximum run count 10 reached for component 'prompt_builder'
iter 0 failed, trying again 0/5
Maximum run count 10 reached for component 'prompt_builder'
iter 1 failed, trying again 1/5


error data = 85, error_rate = 2.82: 100%|█████████▉| 3094/3096 [5:39:29<00:08,  4.31s/it]

Maximum run count 10 reached for component 'prompt_builder'
iter 0 failed, trying again 0/5


error data = 85, error_rate = 2.82: 100%|██████████| 3096/3096 [5:39:39<00:00,  6.58s/it]


In [25]:
with open('new_data.json', 'w') as file:
    json.dump(new_data, file)

with open('error_data.json', 'w') as file:
    json.dump(error_data, file)

In [26]:
with open('new_data.json', 'r') as file:
    newdata1 = json.load(file)
print(len(newdata1))

3011


In [28]:
pprint(newdata1[0])

{'batch': 'S23',
 'cb_url': 'https://www.crunchbase.com/organization/orbio',
 'company_keywords': {'keywords': ['oil and gas industry',
                                   'methane emissions',
                                   'satellite imagery',
                                   'emission factors',
                                   'energy companies',
                                   'regulation',
                                   'investment',
                                   'financial companies',
                                   'Excel-based emission factors']},
 'company_name': 'Orbio Earth',
 'description': 'Orbio uses satellite imagery to track methane emissions from '
                'the oil & gas industry. We sell the data to financial '
                'companies that want to invest into the best performing energy '
                'companies.\r\n'
                '\n'
                'Until today, oil & gas companies use excel-based emission '
                'fac